> Para los productos que contienen en su descripción la palabra “stuff” (sin importar mayúsculas o minúsculas), calcular el peso total de su inventario agrupado por marca, mostrar sólo la marca y el peso total de las 5 más pesadas.

Para esta consulta, vamos a usar los datasets `products` e `inventory_logs`.

Vamos a empezar con lo más sencillo... filtrando y quedandonos con los productos que contienen "stuff" en su descripción.

In [48]:
import pandas as pd

products_df = pd.read_pickle('../data/clean/products.pkl')
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 1 to 1000000
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   product_name    939819 non-null   string        
 1   category_id     1000000 non-null  Int64         
 2   brand           939944 non-null   category      
 3   price           850000 non-null   Float64       
 4   cost            850000 non-null   Float64       
 5   stock_quantity  850000 non-null   Int64         
 6   weight_kg       850000 non-null   Float64       
 7   dimensions      940014 non-null   string        
 8   description     939716 non-null   string        
 9   is_active       1000000 non-null  boolean       
 10  created_at      910329 non-null   datetime64[ns]
dtypes: Float64(3), Int64(2), boolean(1), category(1), datetime64[ns](1), string(3)
memory usage: 84.9+ MB


In [55]:
KEY_WORD = "stuff"

REGEX_FILTER = rf"\b{KEY_WORD}\b"
stuff_products = products_df[
    products_df["description"].str.contains(REGEX_FILTER, case=False, na=False)
]

stuff_products = stuff_products[["brand", "weight_kg"]]
stuff_products.head()

,brand,weight_kg
product_id,,
5,Ikea,31.47
23,La-Z-Boy,46.17
107,Dove,10.12
187,Warner Bros.,17.33
289,Yamaha,10.97


Ahora vamos a intentar reconstruir el inventario actual de cada producto. Para eso, vamos a usar el dataset `inventory_logs`, que tiene un registro de todas las entradas y salidas de inventario por producto con su cantidad y fecha.

In [50]:
df_inventory = pd.read_pickle('../data/clean/inventory_logs.pkl')

df_inventory = df_inventory[['product_id', 'movement_type', 'quantity_change', 'timestamp']]
df_inventory.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400000 entries, 1 to 400000
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   product_id       400000 non-null  Int64         
 1   movement_type    364060 non-null  category      
 2   quantity_change  340000 non-null  Int64         
 3   timestamp        363851 non-null  datetime64[ns]
dtypes: Int64(2), category(1), datetime64[ns](1)
memory usage: 13.4+ MB


Como no podemos operar si no sabemos datos del producto, tipo de movimiento o cantidad, vamos a descartar las filas que tengan datos nulos en esas columnas y de a poco ir simplificando el dataframe. En el caso de los productos, ya tenemos todos como NO nulos así que no hace falta comprobar.

In [51]:
df_inventory = df_inventory.dropna(
    subset=["movement_type", "quantity_change"]
)
df_inventory.info()

<class 'pandas.core.frame.DataFrame'>
Index: 309268 entries, 1 to 400000
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   product_id       309268 non-null  Int64         
 1   movement_type    309268 non-null  category      
 2   quantity_change  309268 non-null  Int64         
 3   timestamp        281384 non-null  datetime64[ns]
dtypes: Int64(2), category(1), datetime64[ns](1)
memory usage: 10.3+ MB


Luego sería ideal simplificar los movimientos a un solo valor real positivo o negativo, para poder sumar todo al final y obtener el inventario actual. Para eso, vamos a crear una nueva columna `quantity_signed` que va a tener el valor de `quantity_change` pero con el signo cambiado si el `movement_type` es "OUT". El otro caso es si el `movement_type` es "ADJUSTMENT", en ese caso vamos a asumir que el valor de `quantity_change` ya tiene el signo correcto y lo dejamos igual.

In [52]:
df_inventory["quantity_signed"] = df_inventory.apply(
    lambda row: -row["quantity_change"]
    if row["movement_type"] == "OUT"
    else row["quantity_change"],
    axis=1,
)

df_inventory = df_inventory[["product_id", "quantity_signed", "timestamp"]]
df_inventory["product_id"] = df_inventory["product_id"].astype(str)

df_inventory.head()

,product_id,quantity_signed,timestamp
log_id,,,
1,996381,228,2023-08-21 04:33:50.446753
4,997550,260,2024-09-03 20:47:37.637720
5,978028,263,2024-10-19 08:55:45.508647
6,990089,6,2025-06-02 12:53:02.444493
7,911658,87,2024-04-03 01:01:52.216227


Ahora que ya tenemos todo reducido a lo que necesitamos, podemos hacer un inner join entre el dataframe de productos y el de inventario, usando `product_id` como clave. Luego a esto lo podemos agrupar por producto y sumar los movimientos para obtener el inventario actual de cada producto.

In [ ]:
merged_products_inventory = pd.merge(
    stuff_products, df_inventory,
    left_index=True, right_on="product_id",
    how="inner"
)

inventory_by_product = merged_products_inventory.groupby(observed=False
    ["product_id", "brand", "weight_kg"]
).agg(
    total_inventory=("quantity_signed", "sum")
).reset_index()

inventory_by_product

/tmp/ipykernel_20876/362923857.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  inventory_by_product = merged_products_inventory.groupby(
